In [1]:
#импорт библиотек
import pandas as pd

#для визуализации
import plotly.express as px
import dash

from dash import Dash, dcc, html, Input, Output

In [2]:
#считываю датасет
games = pd.read_csv('games.csv')
games

,Name,Platform,Year_of_Release,Genre,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,NaN,NaN,NaN
...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,NaN,NaN,NaN


In [3]:
#фильтрую, оставляя данные после 2000, и исключаю строки с пропусками
games = games.query('Year_of_Release >= 2000').dropna() 

In [4]:
#проверяю типы данных в колонках
games.dtypes

Name                object
Platform            object
Year_of_Release    float64
Genre               object
Critic_Score       float64
User_Score          object
Rating              object
dtype: object

In [5]:
#значения годов - в даты
games["Year_of_Release"] = pd.to_datetime(games["Year_of_Release"], format='%Y').dt.year

In [6]:
#перевожу User_Score в численный тип
games.User_Score = pd.to_numeric(games.User_Score, errors='coerce')

In [7]:
games_grp = games.groupby(['Genre', 'Platform', 'Year_of_Release', 'Rating'], as_index = False).agg({'Name':'count'}).rename(columns = {'Name':'Number'})
games_grp

,Genre,Platform,Year_of_Release,Rating,Number
0,Action,3DS,2011,E,2
1,Action,3DS,2011,E10+,9
2,Action,3DS,2011,M,1
3,Action,3DS,2011,T,5
4,Action,3DS,2012,E,4
...,...,...,...,...,...
2167,Strategy,XB,2005,M,1
2168,Strategy,XB,2005,T,2
2169,Strategy,XB,2006,M,1
2170,Strategy,XOne,2013,E,1


In [8]:
#pip install dash-bootstrap-components
import dash_bootstrap_components as dbc

In [9]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)
app = dash.Dash(external_stylesheets=[dbc.themes.ZEPHYR])

app.layout = html.Div([
    html.Div([
        html.H2("Games market"),
        html.P(["Interactive dashboard for monitoring the state of the gaming industry.",html.Br(),"Depending on the selected genres, ratings, and release years, you'll see the number of selected games, game released by year and platform, and player and critic ratings."]),
        
        # фильтр жанра
        html.Div([
            html.P("Choose genre:", style={'font-size': '18px'}),
            dcc.Dropdown(
                id='Genre',
                options=games_grp['Genre'].unique(),
                value=games_grp['Genre'].unique()[[0, 1]],
                multi=True
            ),
        ], style={'width': '48%', 'display': 'inline-block'}),
    
        # фильтр рейтингов
        html.Div([
            html.P("Choose rating:", style={'font-size': '18px'}),
            dcc.Dropdown(
                id='Rating',
                options=games_grp['Rating'].unique(),
                value=games_grp['Rating'].unique()[[0, 1]],
                multi=True
            ),
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
    ]),
            
    #интерактивный текст
    html.Br(),
    html.Div(id='textarea-example-output', style={'font-size': '20px'}),
    
    html.Div([
        # левый стакед плот
        html.Div([
           # html.H4('Stacked area plot by years and platforms'),
            dcc.Graph(id='indicator-graphic'),
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '0 20'}),
        
        # правый скатер плот    
        html.Div([
           # html.H4('Scatter plot by ratings and genres'),
            dcc.Graph(id="scatter-plot"),
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
    ]),
    
    # слайдер выбора промежутка годов
    html.Div([
        html.P("Choose years:"),
        dcc.RangeSlider(
            games_grp.Year_of_Release.min(), games_grp.Year_of_Release.max(),
            step=1,
            marks={i: f'{i}' for i in range(games_grp.Year_of_Release.min(), games_grp.Year_of_Release.max()+1)},
            value=[2001, 2010],
            id='my-range-slider'
        ),
    ], style={'width': '80%', 'padding-top': '20px','font-size': '18px'}),
    html.Br()
])

In [10]:
@app.callback(
    Output('textarea-example-output', 'children'),
    Output('indicator-graphic', 'figure'),
    Output('scatter-plot', 'figure'),
    Input('Genre', 'value'),
    Input('Rating', 'value'),
    Input('my-range-slider', 'value')
)
def update_output(selected_Genre, selected_Rating, selected_Range):
    df = games_grp
    df1 = df[df['Genre'].isin(selected_Genre)]
    df2 = df1.query('Year_of_Release>=@selected_Range[0] & Year_of_Release<=@selected_Range[1]')
    filtered_df = df2[df2['Rating'].isin(selected_Rating)]
    number = filtered_df.Number.sum()
    text_output = f'Number of games: {number}'
    fig1, fig2 = display_area(selected_Genre, selected_Rating, selected_Range)
    return text_output, fig1, fig2

def display_area(selected_Genre, selected_Rating, selected_Range):
    df = games_grp
    df1 = df[df['Genre'].isin(selected_Genre)]
    df2 = df1.query('Year_of_Release>=@selected_Range[0] & Year_of_Release<=@selected_Range[1]')
    filtered_df = df2[df2['Rating'].isin(selected_Rating)]
    fig1 = px.area(
        filtered_df, x="Year_of_Release", y='Number',
        color="Platform",
        labels={"Number": "Games released", "Year_of_Release": "Year of release"}
    )
    fig1.update_layout(font=dict(size=18))
    
    fig2 = update_scatter_plot(selected_Genre, selected_Rating, selected_Range)
    
    return fig1, fig2

def update_scatter_plot(selected_Genre, selected_Rating, selected_Range):
    df = games
    df1 = df[df['Genre'].isin(selected_Genre)]
    df2 = df1.query('Year_of_Release>=@selected_Range[0] & Year_of_Release<=@selected_Range[1]')
    filtered_df = df2[df2['Rating'].isin(selected_Rating)]
    fig2 = px.scatter(
        filtered_df, x='User_Score', y='Critic_Score',
        color='Genre', 
        labels={'User_Score': 'User Score', 'Critic_Score': 'Critic Score'}
    )
    
    fig2.update_layout(font=dict(size=18))
    
    return fig2

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
